In [1]:
import sys
sys.path.append("C:/Users/parspooya/anaconda3/envs/myNewEnv/Lib/site-packages")

import numpy as np
import pandas as pd
from stable_baselines3.common.vec_env import DummyVecEnv
from stable_baselines3 import A2C
import gym
from gym import spaces
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
import matplotlib.pyplot as plt
from stable_baselines3.common.callbacks import BaseCallback
from stable_baselines3.common.logger import configure

In [2]:
# بارگذاری دیتاست
data = pd.read_excel('Book5.xlsx')

In [3]:
# داده‌های ورودی و خروجی
X = data[['p1','k1','s1','t1','a1','p2','k2','s2','t2','a2','p3','k3','s3','t3','a3','p4','k4','s4','t4','a4','p5','k5','s5','t5','a5'
]].values
y = data['task_variant'].values

In [4]:
# تقسیم داده‌ها به داده‌های آموزشی و تست
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [5]:
# تعریف محیط
class CustomEnv(gym.Env):
    def __init__(self, X, y):
        super(CustomEnv, self).__init__()
        self.X = X
        self.y = y
        self.current_step = 0
        self.action_space = spaces.Discrete(3)  # 3 actions: 0, 1, 2
        self.observation_space = spaces.Box(low=-np.inf, high=np.inf, shape=(25,), dtype=np.float32)
    
    def reset(self):
        self.current_step = 0
        return self.X[self.current_step]
    
    def step(self, action):
        reward = 1 if action + 1 == self.y[self.current_step] else 0
        self.current_step += 1
        done = self.current_step >= len(self.X)
        obs = self.X[self.current_step] if not done else np.zeros(self.observation_space.shape)
        return obs, reward, done, {}
    
    def render(self, mode='human'):
        pass

In [6]:
# تعریف Callback
class CustomCallback(BaseCallback):
    def __init__(self, verbose=0):
        super(CustomCallback, self).__init__(verbose)
        self.episode_lengths = []
        self.losses = []
        self.episode_rewards = []

    def _on_step(self) -> bool:
        if 'loss' in self.locals:
            loss = self.locals['loss'].item()
            self.losses.append(loss)

        if 'dones' in self.locals and self.locals['dones'][0]:
            self.episode_lengths.append(self.locals['infos'][0]['episode']['l'])
            self.episode_rewards.append(self.locals['infos'][0]['episode']['r'])
        return True

    def get_metrics(self):
        return self.losses, self.episode_lengths, self.episode_rewards

In [7]:
del model

NameError: name 'model' is not defined

In [ ]:
# ساخت وکتور محیط و آموزش مدل A2C
env = DummyVecEnv([lambda: CustomEnv(X_train, y_train)])

# تنظیمات مدل A2C
model = A2C(
    'MlpPolicy', 
    env, 
    learning_rate=0.001,   
    gamma=0.99,
    tensorboard_log='./dqn_tensorboard/' ,
    verbose=1
)

# callback = CustomCallback()
# history = model.learn(total_timesteps=100000, log_interval=4, callback=callback)

history = model.learn(total_timesteps=100000, log_interval=4)

# model.save("a2c_25enters")

Using cpu device
Logging to ./dqn_tensorboard/A2C_1
------------------------------------
| time/                 |          |
|    fps                | 468      |
|    iterations         | 4        |
|    time_elapsed       | 0        |
|    total_timesteps    | 20       |
| train/                |          |
|    entropy_loss       | -1.06    |
|    explained_variance | -39      |
|    learning_rate      | 0.001    |
|    n_updates          | 3        |
|    policy_loss        | 1.17     |
|    value_loss         | 1.72     |
------------------------------------
------------------------------------
| time/                 |          |
|    fps                | 418      |
|    iterations         | 8        |
|    time_elapsed       | 0        |
|    total_timesteps    | 40       |
| train/                |          |
|    entropy_loss       | -0.828   |
|    explained_variance | -26.5    |
|    learning_rate      | 0.001    |
|    n_updates          | 7        |
|    policy_loss       

In [11]:
# ارزیابی مدل
env = CustomEnv(X_test, y_test)
obs = env.reset()
correct_predictions = 0
total_predictions = 0

# لیست پیش‌بینی‌ها و برچسب‌های واقعی
all_predictions = []
all_labels = []

while True:
    action, _states = model.predict(obs)
    obs, reward, done, info = env.step(action)
    correct_predictions += reward
    total_predictions += 1
    all_predictions.append(action + 1)  # عمل‌ها از 0، 1، 2 به 1، 2، 3 تبدیل می‌شوند
    all_labels.append(env.y[env.current_step - 1])
    if done:
        obs = env.reset()
        break

# ارزیابی
accuracy = accuracy_score(all_labels, all_predictions)
f1 = f1_score(all_labels, all_predictions, average='weighted')
precision = precision_score(all_labels, all_predictions, average='weighted')
recall = recall_score(all_labels, all_predictions, average='weighted')

print(f'Accuracy: {accuracy*100:.2f}%')
print(f'F1 Score: {f1:.2f}')
print(f'Precision: {precision:.2f}')
print(f'Recall: {recall:.2f}')


Accuracy: 87.75%
F1 Score: 0.87
Precision: 0.88
Recall: 0.88


In [90]:
# tensorboard --logdir ./dqn_tensorboard/